In [1]:
# import Libraries
import yt_dlp
from pydub import AudioSegment
from src.youtube_downloader import YouTubeDownloader
# pip install yt-dlp
# pip install pydub
# pip install moviepy

/opt/anaconda3/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
# pip install --upgrade yt-dlp

# Initialize downloader
downloader = YouTubeDownloader()

# If you need to specify ffmpeg path:
# downloader = YouTubeDownloader(ffmpeg_path='/usr/local/bin/ffmpeg')

In [ ]:
from yt_dlp import YoutubeDL
import sys
import os

def download_youtube_video(url, format_id=None):
    # Ensure ffmpeg is in PATH
    if '/usr/local/bin' not in os.environ['PATH']:
        os.environ['PATH'] += ':/usr/local/bin'

    # Configuración para yt-dlp
    ydl_opts = {
        'format': format_id if format_id else 'bestvideo[ext=mp4][height<=1080]+bestaudio[ext=m4a]/best[ext=mp4]/mp4',
        'outtmpl': '%(title)s.%(ext)s',
        'quiet': False,
        'no_warnings': False,
        'cookiesfrombrowser': ('chrome',),
        'extractor_retries': 5,
        'fragment_retries': 10,
        'retries': 10,
        'file_access_retries': 5,
        'ffmpeg_location': '/usr/local/bin/ffmpeg',  # Explicitly set ffmpeg path
        'http_headers': {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
        },
        'merge_output_format': 'mp4',
        'postprocessors': [{
            'key': 'FFmpegVideoConvertor',
            'preferedformat': 'mp4',
        }]
    }

    # Verify ffmpeg installation
    def check_ffmpeg():
        try:
            import subprocess
            result = subprocess.run(['ffmpeg', '-version'], 
                                  stdout=subprocess.PIPE, 
                                  stderr=subprocess.PIPE)
            return result.returncode == 0
        except Exception:
            return False

    if not check_ffmpeg():
        print('Error: ffmpeg is not properly installed or not found in PATH')
        print('Please ensure ffmpeg is installed and accessible')
        return False

    def try_different_formats(ydl, url):
        formats_to_try = [
            'bestvideo[ext=mp4][height<=1080]+bestaudio[ext=m4a]',
            'best[ext=mp4]',
            '22',
            '18',
            'worstvideo[ext=mp4]+worstaudio[ext=m4a]'
        ]
        
        for fmt in formats_to_try:
            try:
                ydl_opts['format'] = fmt
                with YoutubeDL(ydl_opts) as ydl:
                    print(f'\nTrying format: {fmt}')
                    info = ydl.extract_info(url, download=False)
                    formats = info.get('formats', [])
                    
                    # Filter for MP4 formats only
                    mp4_formats = [f for f in formats if f.get('ext') == 'mp4' or 
                                 (f.get('vcodec') != 'none' and f.get('acodec') != 'none')]
                    
                    if mp4_formats:
                        return True, info
            except Exception as e:
                if '403' not in str(e):  # Only continue if it's not a 403 error
                    raise e
                print(f'Format {fmt} not available, trying next...')
        return False, None

    try:
        # Try to get available formats first
        success, info = try_different_formats(YoutubeDL(ydl_opts), url)
        
        if not success:
            print('Could not find any suitable MP4 format. Trying alternative method...')
            # Try with minimal options but still with ffmpeg path
            ydl_opts = {
                'format': '18/best[ext=mp4]',
                'outtmpl': '%(title)s.%(ext)s',
                'quiet': False,
                'merge_output_format': 'mp4',
                'ffmpeg_location': '/usr/local/bin/ffmpeg'
            }
        
        print(f'\nDescargando: {url}')
        with YoutubeDL(ydl_opts) as ydl:
            result = ydl.download([url])
            
            if result == 0:
                print('Descarga completada exitosamente.')
                # Verify the downloaded file is MP4
                if info and info.get('title'):
                    filename = f"{info['title']}.mp4"
                    if os.path.exists(filename):
                        print(f'Archivo guardado como: {filename}')
                return True
            else:
                print('Error durante la descarga.')
                return False

    except Exception as e:
        print(f'Error al descargar el video: {e}')
        if 'ffmpeg not installed' in str(e).lower():
            print('\nError: ffmpeg is required but not properly configured.')
            print('Please ensure ffmpeg is installed at /usr/local/bin/ffmpeg')
        elif '403' in str(e):
            print('\nError: Access forbidden. This might be due to:')
            print('1. Region restrictions')
            print('2. Age restrictions')
            print('3. Private video')
            print('4. YouTube bot detection')
            print('\nTry:')
            print('1. Logging into YouTube in your browser')
            print('2. Using a VPN')
            print('3. Checking if the video is available in your region')
        return False

# Example: Download a video
url = input('Enter YouTube URL: ')
download_youtube_video(url)


Trying format: bestvideo[ext=mp4][height<=1080]+bestaudio[ext=m4a]
Extracting cookies from chrome
Extracted 3485 cookies from chrome
Extracted 3485 cookies from chrome
[youtube] Extracting URL: https://youtu.be/wkwgED7iV6k
[youtube] wkwgED7iV6k: Downloading webpage
[youtube] Extracting URL: https://youtu.be/wkwgED7iV6k
[youtube] wkwgED7iV6k: Downloading webpage
[youtube] wkwgED7iV6k: Downloading tv client config
[youtube] wkwgED7iV6k: Downloading tv client config
[youtube] wkwgED7iV6k: Downloading player 91e7c654-main
[youtube] wkwgED7iV6k: Downloading player 91e7c654-main
[youtube] wkwgED7iV6k: Downloading tv player API JSON
[youtube] wkwgED7iV6k: Downloading tv player API JSON

Descargando: https://youtu.be/wkwgED7iV6k
Extracting cookies from chrome

Descargando: https://youtu.be/wkwgED7iV6k
Extracting cookies from chrome
Extracted 3485 cookies from chrome
Extracted 3485 cookies from chrome
[youtube] Extracting URL: https://youtu.be/wkwgED7iV6k
[youtube] wkwgED7iV6k: Downloading web

ERROR: You have requested merging of multiple formats but ffmpeg is not installed. Aborting due to --abort-on-error


Error al descargar el video: ERROR: You have requested merging of multiple formats but ffmpeg is not installed. Aborting due to --abort-on-error


False

In [ ]:
# Audio converter from mkv to mp3

from pydub import AudioSegment
import os

# Manually set the ffmpeg and ffprobe paths
os.environ["PATH"] += ":/usr/local/bin"  # Add ffmpeg location to PATH
AudioSegment.ffmpeg = "/usr/local/bin/ffmpeg"
AudioSegment.ffprobe = "/usr/local/bin/ffprobe"

# Check if the paths are being set correctly
from pydub.utils import which
print("ffmpeg path:", which("ffmpeg"))
print("ffprobe path:", which("ffprobe"))

# Path to your MKV file
input_file = "Plank_Workout.mkv"
output_file = "Plank_Workout.mp3"

try:
    # Load the MKV file
    audio = AudioSegment.from_file(input_file)

    # Convert to MP3
    audio.export(output_file, format="mp3", bitrate="192k")
    print(f"Audio converted successfully to {output_file}")
except Exception as e:
    print(f"An error occurred during conversion: {e}")

# Example: Convert video to MP3
input_file = input('Enter video filename to convert: ')
downloader.convert_to_mp3(input_file)

/opt/anaconda3/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


ffmpeg path: /usr/local/bin/ffmpeg
ffprobe path: /usr/local/bin/ffprobe
Audio converted successfully to Plank_Workout.mp3


In [ ]:
# Audio converter mp3 to midi

# Example: Download and convert in one step
url = input('Enter YouTube URL: ')
downloader.process_url(url, convert_to_audio=True, delete_video=False)


